In [0]:
-- step 1: find the first website_pageview_id for relevant sessions
-- step 2: id the landing page of each sessions
-- step 3: counting pageviews for each session, to id bounces
-- step 4: summarizing total sessions and bounced sessions by LP

create temporary table lander_first_created3
select 
min(created_at) as first_created_at,
min(website_pageview_id) as min_pv_id,
CAST( min(created_at) AS DATE) as first_day
from mavenfuzzyfactory.website_pageviews
where pageview_url = '/lander-1' and created_at is not null


CREATE TEMPORARY TABLE first_pageviews_demo3
select 
  wp.website_session_id,
  min(wp.website_pageview_id) as min_pv_id
from workspace.mavenfuzzyfactory.website_pageviews wp
inner join workspace.mavenfuzzyfactory.website_sessions ws
  on wp.website_session_id = ws.website_session_id
  and ws.created_at < '2012-07-28'
  and wp.website_pageview_id >= (select min_pv_id from lander_first_created3)
  and ws.utm_source = 'gsearch'
  and ws.utm_campaign = 'nonbrand'
group by 1;

-- step 2
create temporary table sessions_w_landing_page_demo3
select 
  fpd.website_session_id,
  wp.pageview_url as landing_page
from first_pageviews_demo3 as fpd
join workspace.mavenfuzzyfactory.website_pageviews wp
on fpd.min_pv_id = wp.website_pageview_id
where wp.pageview_url in ('/home', '/lander-1')  

-- step 3
create temporary table bounced_sessions_only3
select 
  swlpd.website_session_id,
  swlpd.landing_page,
  count(distinct wp.website_pageview_id) as bounces
from sessions_w_landing_page_demo3 as swlpd
left join workspace.mavenfuzzyfactory.website_pageviews wp
  on swlpd.website_session_id = wp.website_session_id
group by 1,2
having count(distinct wp.website_pageview_id) = 1

-- step 4
select 
  swlpd.landing_page,
  count(distinct swlpd.website_session_id ) as sessions,
  count(distinct bsod.website_session_id ) as bounced_sessions,
  round(count(distinct bsod.website_session_id )/count(distinct swlpd.website_session_id )*100,2) as bounce_rate
from sessions_w_landing_page_demo3 swlpd
left join bounced_sessions_only3 bsod
  on swlpd.website_session_id = bsod.website_session_id
group by 1
order by 2
